In [1]:
from PAX1000LTM import *
import instrumental
from pylablib.devices import Thorlabs

In [2]:
# connect to rotating stage
Thorlabs.kinesis.BasicKinesisDevice.list_devices()
stage1 = Thorlabs.kinesis.KinesisMotor(str(Thorlabs.kinesis.BasicKinesisDevice.list_devices()[0][0])) # polarimeter
stage2 = Thorlabs.kinesis.KinesisMotor(str(Thorlabs.kinesis.BasicKinesisDevice.list_devices()[1][0])) # NA
stage3 = Thorlabs.kinesis.KinesisMotor(str(Thorlabs.kinesis.BasicKinesisDevice.list_devices()[2][0])) # sample theta
stage4 = Thorlabs.kinesis.KinesisMotor(str(Thorlabs.kinesis.BasicKinesisDevice.list_devices()[3][0])) # sample phi

In [3]:
# positive numbers means clockwise rotation
Unit1 = 1633 # polarimeter corrected, 
Unit2 = 1923 
Unit3 = 1923 # sample theta corrected 
Unit4 = 1923 # sample phi corrected, speed = 20000, acceleration = 13000, ~15.5 s for 90 degree

In [4]:
# connect to polarimeter
resource = b'USB0::0x1313::0x8031::M00766101::INSTR'

polarimeter = Polarimeter(resource, 9, 0.000000532) # measureMode:9, wavelength: 532nm
polarimeter.connectDevice()

Number of available devices: 1
The device's availability: True, model: b'M00766101', manufacturer: b'M00766101', serial number: b'M00766101'
Device initialization succeed!
The measureMode is set to be: 9
The wavelength is set to be: 5.32e-07 m


In [5]:
# parameter configuration
deltaPhi = 90
deltaTheta = 45
deltaPolarimeter = 160

stepTheta = 3
stepPola = 2

Unit1 = -Unit1 # polarimeter
Unit3 = -Unit3 # theta
Unit4 = Unit4 # phi

In [6]:
# set current position as the reference
stage1.set_position_reference()
stage4.set_position_reference()
stage3.set_position_reference()

stage4.setup_velocity(min_velocity=0, acceleration=30000, max_velocity=13000)

TVelocityParams(min_velocity=0.0, acceleration=29991.497285664078, max_velocity=12999.996542930603)

In [7]:
# initialize the measure class
ltm = PolarimeterLTM(polarimeter.handler, stage4, deltaPhi*Unit4, 16, 0.05)

In [8]:
def main():
    angle_pola = 0
    angle_theta = 0
    pola_input = 'RCW'
    os.makedirs(os.path.join(cwd, pola_input), exist_ok=True)
    for theta in range(angle_theta, deltaTheta+1, stepTheta):
        for pola in range(0, deltaPolarimeter+1, stepPola):
            print("running for theta {}, pola {}".format(theta, pola))
            ltm.run(os.path.join(cwd,  "{}/theta{}_pola{}".format(pola_input, theta, pola)))
            if angle_pola + stepPola > deltaPolarimeter:
                stage1.move_to(0)
                angle_pola = 0
            else:
                stage1.move_by(Unit1*stepPola)
                angle_pola += stepPola 
            stage1.wait_move()
            time.sleep(1) # wait for polarimeter to stablize
        if angle_theta + stepTheta > deltaTheta:
            stage3.move_to(0)
            angle_theta = 0
        else:
            stage3.move_by(Unit1*stepTheta)
            angle_theta += stepTheta
        stage3.wait_move()
        time.sleep(1) # wait for polarimeter to stablize
    polarimeter.closeDevice()

In [15]:
# run
main()

running for theta 0, pola 0
Measure start...
Measure stopped.
will now write to file...
CSV file saved to C:\Users\Nanochiral_2\Documents\Python Scripts\Yuan\RCW/theta0_pola0.csv
running for theta 0, pola 2
Measure start...
Measure stopped.
will now write to file...
CSV file saved to C:\Users\Nanochiral_2\Documents\Python Scripts\Yuan\RCW/theta0_pola2.csv


KeyboardInterrupt: 

In [11]:
polarimeter.closeDevice()

In [9]:
stage1.move_by(Unit1*70) # polarimeter

In [10]:
stage4.move_by(Unit4*90) # sample phi

In [16]:
stage1.move_to(0)
stage4.move_to(0)
stage3.move_to(0)